# Subsistema de Gestión de Usuarios y Perfiles basado en MongoDB

Este notebook implementa el Subsistema de **Usuarios y Perfiles** de Duolingo utilizando **MongoDB** como motor principal de almacenamiento.

A diferencia de bases relacionales tradicionales, MongoDB permite almacenar perfiles completos del usuario dentro de un **único documento**, lo que optimiza la lectura de la pantalla principal del alumno.

El sistema se ejecuta sobre un clúster compuesto por:
- Un router **mongos**
- Tres shards con réplica (sharding + replica sets para alta disponibilidad)
- Un config server replicado

---

## Objetivo del Notebook

El objetivo es demostrar los principales patrones de acceso del subsistema, cada uno implementado con operaciones atómicas y consultas óptimas:

### 1. Creación de usuarios y flexibilidad del esquema
- Documentos con estructura heterogénea.
- Campos opcionales.
- Subdocumentos embebidos.

### 2. Carga completa de perfil (lectura crítica)
- Consulta única (`find_one`) que retorna avatar, progreso, privacidad y cursos.

### 3. Actualización de progreso y racha
- Uso de operadores atómicos `$inc` y `$set`.

### 4. Gestión de privacidad
- Cambios sobre subdocumentos.
- Justificación de índices.

### 5. Gestión de suscripción (Plus)
- Subdocumento `suscripcion` embebido.

### 6. Gestión de amigos embebidos
- Uso de `$addToSet` y `$pull`.

### 7. Transacciones multi-etapa
- Inscripción a un curso + inicialización del progreso.

---

# Modelo de Datos (Documentos BSON)

El modelo central es la colección `usuarios`, cuya estructura general es:

```
{
  _id: <username>,
  username: <string>,
  email: <string>,
  password_hash: <string>,
  fecha_registro: <datetime>,
  2fa_enabled: <bool>,
  avatar: <string | null>,
  bio: <string | null>,
  racha_actual: <int>,
  total_xp: <int>,
  nivel_actual: <int>,

  cursos: [
    {
      idioma_id: <string>,
      xp_curso: <int>,
      unidades_completadas: <int>
    }, ...
  ],

  ajustes: {
    privacidad_perfil: "publico" | "privado" | ...,
    permitir_amigos: <bool>,
    notificaciones: {
      email: <bool>,
      push: <bool>
    }
  },

  suscripcion: {
    es_premium: <bool>,
    fecha_vencimiento: <datetime | null>,
    plan: "mensual" | "anual" | null
  },

  friends_ids: [ <username>, ... ]
}
```

Este formato permite almacenar toda la información del usuario en un **único documento autocontenido**, reduciendo la latencia de lectura.

---

# Archivo auxiliar: `user_setup.py`

Toda la lógica del subsistema (creación de usuarios, índices, transacciones, actualizaciones atómicas, etc.) se encuentra en `user_setup.py`.

La notebook importa estas funciones y se concentra en demostrar los patrones de acceso del sistema.

In [1]:
%pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 12.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 11.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
from user_profile_setup import *

client = get_client("mongodb://mongos:27017")
col = get_users_collection(client)

ensure_indexes(col)
print("Conectado a MongoDB desde mongos.")

Conectado a MongoDB desde mongos.


# Índices en MongoDB

Los índices utilizados reflejan directamente los patrones de acceso especificados en la documentación:

1. Búsqueda por **username** -> índice único.
2. Búsqueda por **email** -> índice único.
3. Consultas por privacidad -> índice en `ajustes.privacidad_perfil`.
4. Consultas por notificaciones -> índice en `ajustes.notificaciones.email`.

Estos índices permiten rendimiento óptimo en búsquedas críticas para la plataforma.

In [ ]:
list(col.list_indexes())

# Patrón 1: Flexibilidad del Esquema (schema-less)

En este patrón se demuestra cómo MongoDB permite almacenar usuarios con estructuras heterogéneas, sin necesidad de actualizar un esquema global.

Se crean:
- Usuario completo
- Usuario minimalista
- Usuario con múltiples cursos inicializados y privacidad personalizada

In [ ]:
create_user(
    col,
    username="ana",
    email="ana@mail.com",
    password_hash="h123",
    bio="Learning French",
    cursos=[{"idioma_id": "fr", "xp_curso": 120, "unidades_completadas": 4}],
)

create_user(
    col,
    username="leo",
    email="leo@mail.com",
    password_hash="h456",
)

create_user(
    col,
    username="mia",
    email="mia@mail.com",
    password_hash="h789",
    cursos=[
        {"idioma_id": "en", "xp_curso": 40, "unidades_completadas": 1},
        {"idioma_id": "pt", "xp_curso": 0, "unidades_completadas": 0},
    ],
    privacidad={"privacidad_perfil": "privado", "permitir_amigos": False},
)

In [ ]:
list(col.find({}, {"_id": 0}))

# Patrón 2: Carga completa del perfil

La operación principal del sistema. Al iniciar Duolingo, el cliente requiere obtener **toda la información del perfil** en una sola consulta.

- Datos visuales
- Progreso
- Cursos
- Ajustes de privacidad
- Suscripción
- Lista de amigos


In [ ]:
create_user(
    col,
    username="sofia",
    email="sofia@mail.com",
    password_hash="hash",
    cursos=[
        {"idioma_id": "en", "xp_curso": 200, "unidades_completadas": 5},
        {"idioma_id": "it", "xp_curso": 50, "unidades_completadas": 1},
    ],
)

In [3]:
get_profile(col, "sofia")

{'_id': 'sofia',
 'username': 'sofia',
 'email': 'sofia@mail.com',
 'password_hash': 'hash',
 'fecha_registro': datetime.datetime(2025, 12, 4, 20, 4, 17, 48000),
 '2fa_enabled': False,
 'avatar': None,
 'bio': None,
 'racha_actual': 1,
 'total_xp': 15,
 'nivel_actual': 1,
 'cursos': [{'idioma_id': 'en', 'xp_curso': 215, 'unidades_completadas': 5},
  {'idioma_id': 'it', 'xp_curso': 50, 'unidades_completadas': 1},
  {'idioma_id': 'jp', 'xp_curso': 0, 'unidades_completadas': 0}],
 'ajustes': {'privacidad_perfil': 'privado',
  'permitir_amigos': True,
  'notificaciones': {'email': True, 'push': True}},
 'suscripcion': {'es_premium': True,
  'fecha_vencimiento': datetime.datetime(2026, 1, 3, 20, 5, 34, 976000),
  'plan': 'mensual'},
 'friends_ids': ['ana', 'leo'],
 'ultima_actividad': datetime.datetime(2025, 12, 4, 20, 6, 55, 70000)}

# Patrón 3: Actualización de progreso y racha

Cuando el usuario completa una lección:
- Se incrementa el XP global.
- Se actualiza el XP del curso correspondiente.
- Se incrementa la racha.
- Se registra la última actividad.

MongoDB permite realizar todo en una única operación atómica sobre un solo documento.

In [ ]:
update_progress_and_streak(col, "sofia", "en", 15)

In [ ]:
get_profile(col, "sofia")

# Patrón 4: Privacidad y configuración

Los microservicios deben consultar y modificar la privacidad del usuario constantemente.
Esto justifica el índice en:

- `ajustes.privacidad_perfil`

In [ ]:
update_privacy(col, "sofia", "privado")

In [ ]:
get_profile(col, "sofia")["ajustes"]

# Patrón 5: Gestión de suscripción (Plus)

El subdocumento `suscripcion` almacena:
- Estado premium
- Fecha de vencimiento
- Plan contratado

In [ ]:
activate_plus(col, "sofia", plan="mensual")

In [ ]:
get_profile(col, "sofia")["suscripcion"]

# Patrón 6: Lista de amigos embebida

Para acceso rápido, la lista de amigos se almacena embebida en el documento del usuario.

In [ ]:
add_friend(col, "sofia", "ana")
add_friend(col, "sofia", "leo")

In [ ]:
get_profile(col, "sofia")["friends_ids"]

# Patrón 7: Transacción multi-etapa (inscripción a curso)

La inscripción a un curso requiere múltiples actualizaciones que deben mantenerse consistentes. 
Se utiliza una **transacción de MongoDB** para garantizar atomicidad.

In [ ]:
enroll_and_init_course(client, "sofia", "jp")

In [ ]:
get_profile(col, "sofia")["cursos"]

# Conclusiones y relación con los requerimientos no funcionales

Este notebook demuestra cómo MongoDB cumple los requisitos del subsistema:

1. **Flexibilidad del esquema**
   - Documentos heterogéneos.
   - Campos opcionales y subdocumentos embebidos.

2. **Alto rendimiento de lectura**
   - La carga del perfil completo es una única operación.

3. **Escalabilidad horizontal**
   - Sharding basado en `_id`.

4. **Disponibilidad y tolerancia a fallos**
   - Replica sets en cada shard.

5. **Actualizaciones atómicas**
   - Progreso, racha y privacidad.

6. **Transacciones cuando es necesario**
   - Inscripción a cursos.

7. **Índices alineados a los patrones de acceso**
   - Username, email, privacidad, notificaciones.

En conjunto, el subsistema ilustra cómo MongoDB funciona como motor eficiente para gestionar perfiles, progreso, preferencias y estructura social del usuario.